In [1]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

df = pd.read_csv('googleplaystore.csv.csv')

# current time to check
now = datetime.now()
current_hour = now.hour

# Only show graph between 1 PM (13:00) and 2 PM (14:00) IST
if current_hour >= 13 and current_hour < 14:
    
    # Apply filters
    filtered_df = df[
        (df['Installs'] >= 10000) & 
        (df['Revenue'] >= 10000) &
        (df['Android Ver'] > 4.0) &
        (df['Size'] > 15000000) &  # 15 M in bytes
        (df['Content Rating'] == 'Everyone') &
        (df['App'].str.len() <= 30)
    ]
    
    # Get top 3 categories by total installs
    top_3_categories = filtered_df.groupby('Category')['Installs'].sum().nlargest(3).index.tolist()
    
    # Filter for top 3 categories only
    top_3_df = filtered_df[filtered_df['Category'].isin(top_3_categories)]
    
    # Calculate averages for Free vs Paid apps
    free_paid_stats = top_3_df.groupby(['Category', 'Type']).agg({
        'Installs': 'mean',
        'Revenue': 'mean'
    }).reset_index()
    
    # Free and Paid data separately
    free_apps = free_paid_stats[free_paid_stats['Type'] == 'Free']
    paid_apps = free_paid_stats[free_paid_stats['Type'] == 'Paid']
    
    # dual-axis chart
    fig = go.Figure()
    
    # Free Apps - Average Installs -bars
    fig.add_trace(go.Bar(
        x=free_apps['Category'],
        y=free_apps['Installs'],
        name='Free Apps - Avg Installs',
        marker_color='lightgreen',
        yaxis='y'
    ))
    
    # Paid Apps - Average Installs -bars
    fig.add_trace(go.Bar(
        x=paid_apps['Category'],
        y=paid_apps['Installs'],
        name='Paid Apps - Avg Installs',
        marker_color='darkgreen',
        yaxis='y'
    ))
    
    # Free Apps - Average Revenue -line
    fig.add_trace(go.Scatter(
        x=free_apps['Category'],
        y=free_apps['Revenue'],
        name='Free Apps - Avg Revenue',
        mode='lines+markers',
        marker_color='orange',
        line=dict(width=3),
        yaxis='y2'
    ))
    
    # Paid Apps - Average Revenue -line
    fig.add_trace(go.Scatter(
        x=paid_apps['Category'],
        y=paid_apps['Revenue'],
        name='Paid Apps - Avg Revenue',
        mode='lines+markers',
        marker_color='red',
        line=dict(width=3),
        yaxis='y2'
    ))
    
    # Update layout with dual axes
    fig.update_layout(
        title='Top 3 Categories: Average Installs and Revenue (Free vs Paid)',
        xaxis=dict(title='Category'),
        yaxis=dict(
            title='Average Installs',
            side='left'
        ),
        yaxis2=dict(
            title='Average Revenue ($)',
            overlaying='y',
            side='right'
        ),
        barmode='group',
        height=600,
        showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1,
            xanchor="left",
            x=1.15
        )
    )
    
    
    fig.show()
    
else:
    print("This graph is only available between 1 PM and 2 PM IST")
    print(f"Current time: {now.strftime('%I:%M %p')}")

This graph is only available between 1 PM and 2 PM IST
Current time: 08:55 AM
